# AI Exam

An advanced aquatic drone, is deployed to collect critical data on marine biodiversity in a coastal region. The drone starts at point $S$, located near the shore, and must navigate to point $G$, a designated marine research site rich in coral reefs and sea life. Along the way, the drone must carefully maneuver through dynamic underwater environments, avoiding hazards and optimizing its energy usage.

The environment includes:  
1. **(O) Open Water:** Normal movement; no additional challenges.  
2. **(C) Currents:** Areas where the drone's movement is influenced by ocean currents, potentially pushing it off course.  
3. **(F) Seaweed Forests:** Dense vegetation that slows down the drone, incurring extra energy costs per move.    
5. **(E) Energy Stations:** Specific points where the drone can recharge its battery, reducing the total cost navigation.  


<img src="images/env_ex2.png" style="zoom: 20%;"/>



### Environment Details:

- **Grid Representation:** The environment is represented as in the above image (a grid 10x10).  
- $S$ - Start state: The drone's starting point at (0, 0).  
- $G$ - Goal state: The marine research site at (9, 7), providing a large positive reward +20.0 and ending the episode.  
- **Movement Costs:** Each move has a default energy cost of -0.04.  
- **Hazards:**  
  - **Strong Currents:** Entering a zone with current results in a stochastic movement:  
    - 80% chance to move as intended.  
    - 10% chance to be pushed one cell in the left direction perpendicular to the desired movement.  
    - 10% chance to be pushed one cell in the right direction perpendicular to the desired movement. 
  - **Seaweed Forests:** Entering these zones incurs an additional -0.2 reward penalty with respect to the standard movement cost (i.e., a total -0.24 penalty).

- **Energy Stations:** Provide a +1.0 reward when visited (however reaching these cells may require the agent to move far from the goal).  

You can use the following code to explore better the environment

In [5]:
import os, sys 
import tqdm

module_path = os.path.abspath(os.path.join('tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym, envs
from utils.ai_lab_functions import *
import numpy as np
from timeit import default_timer as timer
from tqdm import tqdm as tqdm

env_name = 'AquaticEnv-v0'
env = gym.make(env_name)

env.render()

print("\nActions encoding: ", env.actions)

# Remember that you can know the type of a cell whenever you need by accessing the grid element of the environment:
print("Cell type of start state: ",env.grid[env.startstate])
print("Cell type of goal state: ",env.grid[env.goalstate])
state = 3 # forest
print(f"Cell type of cell {env.state_to_pos(state)}: ",env.grid[state])
state = 12 # corrent
print(f"Cell type of cell {env.state_to_pos(state)}: ",env.grid[state])
state = 17 # energy station
print(f"Cell type of cell {env.state_to_pos(state)}: ",env.grid[state])

[['S' 'O' 'O' 'F' 'F' 'F' 'F' 'O' 'O' 'O']
 ['O' 'F' 'C' 'C' 'C' 'O' 'F' 'E' 'F' 'O']
 ['O' 'O' 'F' 'F' 'F' 'O' 'F' 'F' 'F' 'C']
 ['F' 'C' 'F' 'F' 'E' 'C' 'F' 'O' 'F' 'C']
 ['F' 'C' 'F' 'F' 'F' 'C' 'F' 'O' 'F' 'C']
 ['F' 'E' 'F' 'O' 'O' 'O' 'F' 'E' 'F' 'C']
 ['O' 'O' 'O' 'O' 'O' 'O' 'F' 'F' 'F' 'C']
 ['O' 'F' 'F' 'F' 'O' 'O' 'O' 'F' 'F' 'C']
 ['O' 'O' 'O' 'O' 'F' 'F' 'F' 'F' 'F' 'C']
 ['F' 'F' 'F' 'O' 'O' 'O' 'O' 'G' 'O' 'F']]

Actions encoding:  {0: 'L', 1: 'R', 2: 'U', 3: 'D'}
Cell type of start state:  S
Cell type of goal state:  G
Cell type of cell (0, 3):  F
Cell type of cell (1, 2):  C
Cell type of cell (1, 7):  E


In [45]:
#Action encoding
print("\nActions encoding: ", env.actions)

# Remember that you can know the type of a cell whenever you need by accessing the grid element of the environment:
print("Cell type of start state: ",env.grid[env.startstate])
print("Cell type of goal state: {} Reward: {}".format(env.grid[env.goalstate],env.RS[env.goalstate]))

state = 1 # normal state
print(f"\nCell type of cell {env.state_to_pos(state)}: ",env.grid[state])
print(f"Probability of effectivelty performing action {env.actions[0]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[0], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[1]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[1], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[2]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[2], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[3]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[3], state+1]}")

state = 13 # state with stochastic transitions
print(f"\nCell type of cell {env.state_to_pos(state)}: ",env.grid[state])
print(f"Probability of effectivelty performing action {env.actions[0]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[0], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[1]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[1], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[2]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[2], state+1]}")
print(f"Probability of effectivelty performing action {env.actions[3]} from cell {env.state_to_pos(state)} to cell {env.state_to_pos(state+1)}: {env.T[state, list(env.actions.keys())[3], state+1]}")



Actions encoding:  {0: 'L', 1: 'R', 2: 'U', 3: 'D'}
Cell type of start state:  S
Cell type of goal state: G Reward: 20.0

Cell type of cell (0, 1):  O
Probability of effectivelty performing action L from cell (0, 1) to cell (0, 2): 0.0
Probability of effectivelty performing action R from cell (0, 1) to cell (0, 2): 1.0
Probability of effectivelty performing action U from cell (0, 1) to cell (0, 2): 0.0
Probability of effectivelty performing action D from cell (0, 1) to cell (0, 2): 0.0

Cell type of cell (1, 3):  C
Probability of effectivelty performing action L from cell (1, 3) to cell (1, 4): 0.0
Probability of effectivelty performing action R from cell (1, 3) to cell (1, 4): 0.8
Probability of effectivelty performing action U from cell (1, 3) to cell (1, 4): 0.1
Probability of effectivelty performing action D from cell (1, 3) to cell (1, 4): 0.1


#### Q1. Find an optimal solution to this problem by using the approach that you think is most appropriate. Motivate your choice

In [41]:
env_name = 'AquaticEnv-v0'
env = gym.make(env_name)

def acquaticValueIteration(env, maxIterazione = 500, discount = 0.9, maxDifferenza = 1e-3):
    """
    Args:
        env: ambiente
        maxIterazione: numero massimo di iterazioni
        discount: 
        maxDifferenza: limite superiore da confrontare con delta

    Returns:
        policy: vettore contenente le azioni contentute nel rispettivo stato identificato da "i"
    """

    U_1 = [0 for _ in range(env.observation_space.n)] #array contenente i valori degli stati aggiornati
    U = U_1.copy()  #array contenente i valori dei vecchi stati
    cont = 0

    while True:
        cont += 1
        delta = 0
        U = U_1.copy()

        for state in range(0, env.observation_space.n):
            if env.grid[state] == "G":
                U_1[state] = env.RS[state]
            else:
                insiemePunteggiAzioni = [0 for _ in range(0, env.action_space.n)]
                
                for action in range(0, env.action_space.n):
                    for nextState in range (0, env.observation_space.n):
                        insiemePunteggiAzioni[action] += env.T[state, action, nextState] * U[nextState]
                    
                U_1[state] = env.RS[state] + discount * max(insiemePunteggiAzioni)
            
            delta = max(delta, abs(U_1[state] - U[state]))
        
        if delta < maxDifferenza * (1 - discount) / discount or cont >= maxIterazione:
            break

    return(values_to_policy(np.asarray(U), env), U)

In [42]:
env_name = 'AquaticEnv-v0'
env = gym.make(env_name)

def acquaticPolicyIteration(env, maxIterazione = 500, discount = 0.9, maxDifferenza = 1e-3):
    """
    Args:
        env: ambiente
        maxIterazione: numero massimo di iterazioni
        discount: 
        maxDifferenza: limite superiore da confrontare con delta
        
    Returns:
        policy: vettore contenente le azioni contentute nel rispettivo stato identificato da "i"
    """

    policy = [0 for _ in range(env.observation_space.n)]
    
    U_1 = [0 for _ in range(env.observation_space.n)] #array contenente i valori degli stati aggiornati
    U = U_1.copy()  #array contenente i valori dei vecchi stati
    def argmax(valore):
        return max(range(len(valore)), key = lambda x: valore[x])

    cont = 0
    while True:
        contIterazione = 0
        while True:
            contIterazione += 1
            delta = 0
            U = U_1.copy()

            for state in range(0, env.observation_space.n):
                if env.grid[state] == "G":
                    U_1[state] = env.RS[state]
                else:
                    u_somma = 0
                    
                    for nextState in range (0, env.observation_space.n):
                        u_somma += env.T[state, policy[state], nextState] * U[nextState]
                    
                    U_1[state] = env.RS[state] + discount * u_somma
                
                delta = max(delta, abs(U_1[state] - U[state]))
            
            if delta < maxDifferenza * (1 - discount) / discount or contIterazione >= maxIterazione:
                break
        
        U = U_1.copy()
        unchanged = True
        cont += 1

        for state in range(0, env.observation_space.n):
            insiemePunteggiAzioni = [0 for _ in range(0, env.action_space.n)]
        
            for action in range(0,env.action_space.n):
                for nextState in range(0, env.observation_space.n):
                    insiemePunteggiAzioni[action] += env.T[state, action, nextState] * U[nextState]
            
            sommaPolicy = 0
            for nextState in range(0, env.observation_space.n):
                sommaPolicy += env.T[state, policy[state], nextState] * U[nextState]

            if max(insiemePunteggiAzioni) > sommaPolicy:
                policy[state] = argmax(insiemePunteggiAzioni)
                unchanged = False

        if unchanged or cont >= maxIterazione:
            break
    
    return(values_to_policy(np.asarray(U), env), U)


You can visualize and check your solution using the following code:

In [40]:
print("Metodo: Value Iteration")

t = timer()
solution, _ = acquaticValueIteration(env)
visual_solution = np.vectorize(env.actions.get)(solution.reshape(env.rows, env.cols)) 

print(visual_solution)
print("\nTempo di esecuzione: \n{}".format(round(timer() - t, 4)))

print("\nMetodo: Policy Iteration")

t1 = timer()
solution, _ = acquaticPolicyIteration(env)
visual_solution = np.vectorize(env.actions.get)(solution.reshape(env.rows, env.cols)) 

print(visual_solution)
print("\nTempo di esecuzione: \n{}".format(round(timer() - t1, 4)))

Metodo: Value Iteration
[['D' 'D' 'D' 'D' 'R' 'D' 'R' 'D' 'L' 'L']
 ['D' 'D' 'R' 'R' 'R' 'R' 'R' 'D' 'L' 'L']
 ['R' 'D' 'R' 'R' 'D' 'R' 'R' 'D' 'L' 'L']
 ['D' 'D' 'R' 'R' 'D' 'R' 'R' 'D' 'L' 'L']
 ['D' 'D' 'R' 'D' 'D' 'D' 'R' 'D' 'L' 'L']
 ['R' 'R' 'R' 'R' 'R' 'R' 'R' 'D' 'L' 'L']
 ['D' 'R' 'R' 'R' 'R' 'D' 'D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'D' 'R' 'R' 'D' 'D' 'D' 'D']
 ['R' 'R' 'R' 'D' 'D' 'D' 'D' 'D' 'D' 'D']
 ['R' 'R' 'R' 'R' 'R' 'R' 'R' 'L' 'L' 'L']]

Tempo di esecuzione: 
0.709

Metodo: Policy Iteration
[['D' 'D' 'D' 'D' 'R' 'D' 'R' 'D' 'L' 'L']
 ['D' 'D' 'R' 'R' 'R' 'R' 'R' 'D' 'L' 'L']
 ['R' 'D' 'R' 'R' 'D' 'R' 'R' 'D' 'L' 'L']
 ['D' 'D' 'R' 'R' 'D' 'R' 'R' 'D' 'L' 'L']
 ['D' 'D' 'R' 'D' 'D' 'D' 'R' 'D' 'L' 'L']
 ['R' 'R' 'R' 'R' 'R' 'R' 'R' 'D' 'L' 'L']
 ['D' 'R' 'R' 'R' 'R' 'D' 'D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'D' 'R' 'R' 'D' 'D' 'D' 'D']
 ['R' 'R' 'R' 'D' 'D' 'D' 'D' 'D' 'D' 'D']
 ['R' 'R' 'R' 'R' 'R' 'R' 'R' 'L' 'L' 'L']]

Tempo di esecuzione: 
2.6671


Confronto tra gli algoritmi implementati:

Il *Value Iteration* converge alla funzione valore ottimale V∗ solo in modo asintotico: ci si ferma quando il cambiamento tra due iterazioni è inferiore alla soglia maxDifferenza, tuttavia non si ha mai la certezza matematica assoluta di aver raggiunto il valore esatto, ma solo un'approssimazione molto stretta.

Il *Policy Iteration* converge alla policy ottimale in un numero finito di iterazioni. La Policy Iteration esplora sistematicamente le opzioni fino a trovare quella perfetta.
Il calcolo costa però al computer maggior tempo e maggior spazio, in correlazione a quanto grande è il problema da affrontare o l'ambiente preso in considerazione.

#### Analyze the solution returned by your approach and comment on whether the solution passes by at least two charging stations to reach the coral for every possible execution.

Nella Value Iteration, data la stocasticità della transizione nelle Correnti, solo nel caso la sonda si trovi nella cella (4,1)  vi è un 10% di probabilità utilizzando la action migliore, ovvero DOWN, di essere spinto verso RIGHT, intraprendendo un percorso che non prevede il passaggio in nessuna seconda colonna di Energia. Di conseguenza non esiste nessuna certezza di passare in due colonne per ogni esecuzione del programma.

Analogamente, nella Policy Iteration, avendo trovao la medesima soluzione del Value Iteration, vi sarà la stessa probabilità di essere spinti via dalla corrente e di intraprendere un percorso che non prevede il passaggio per due colonne di Energia.
